In [6]:
import ast
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Define the patterns and solutions for different groups and subcategories
MISUSE_PATTERNS = {
    'code_level_misuse': {
        'weak_cryptography': {
            'md5': 'MD5 is considered weak. Use SHA-256 or stronger.',
            'sha1': 'SHA-1 is considered weak. Use SHA-256 or stronger.',
            'rc4': 'RC4 is considered insecure. Use AES or other secure algorithms.',
            'des': 'DES is considered weak. Use AES or other secure algorithms.',
        },
        'coding_implementation_bugs': {
            'hardcoded_keys': 'Hardcoded cryptographic keys detected. Use environment variables or secure key management systems.',
        },
        'bad_randomness_handling': {
            'random': 'Insecure random number generation detected. Use os.urandom or secrets module for secure random numbers.',
        }
    },
    'design_flaws': {
        'program_design_flaws': {
            'custom_encryption': 'Custom encryption implementations are risky. Use established libraries and algorithms.',
        },
        'improper_certificate_validation': {
            'cert_validation': 'Improper certificate validation detected. Ensure all certificates are properly validated.',
        },
        'public_key_cryptographic_issues': {
            'weak_key_sizes': {
                'RSA': 'RSA keys should be at least 2048 bits. Prefer 3072 or 4096 bits for strong security.',
            }
        }
    },
    'insecure_architecture': {
        'iv_nonce_management': {
            'ECB': 'ECB mode is insecure. Use CBC, GCM, or other secure modes.',
        },
        'poor_key_management': {
            'key_management': 'Poor key management practices detected. Use secure key storage solutions and rotate keys regularly.',
        },
        'crypto_architecture_infrastructure': {
            'deprecated_libraries': {
                'pycrypto': 'PyCrypto is deprecated and insecure. Use PyCryptodome as a replacement.',
            }
        }
    }
}

# Sample Python code to analyze
SAMPLE_CODE = """
import hashlib
import random

key = 'hardcoded_secret_key'

def encrypt(data):
    hash_object = hashlib.md5(data.encode())
    return hash_object.hexdigest()

def bad_random_number():
    return random.random()

def use_ecb_mode():
    from Crypto.Cipher import AES
    cipher = AES.new(key.encode(), AES.MODE_ECB)
    return cipher.encrypt(data.encode())

def custom_encryption(data):
    # Custom encryption logic
    return data[::-1]
"""

# Function to analyze the code
def analyze_code(source_code):
    tree = ast.parse(source_code)
    findings = []

    class CryptoAnalyzer(ast.NodeVisitor):
        def visit_Import(self, node):
            for alias in node.names:
                if alias.name in MISUSE_PATTERNS['insecure_architecture']['crypto_architecture_infrastructure']['deprecated_libraries']:
                    findings.append(('insecure_architecture', 'crypto_architecture_infrastructure', alias.name, MISUSE_PATTERNS['insecure_architecture']['crypto_architecture_infrastructure']['deprecated_libraries'][alias.name], node.lineno))
            self.generic_visit(node)

        def visit_ImportFrom(self, node):
            if node.module:
                module_name = node.module.split('.')[0]
                if module_name in MISUSE_PATTERNS['insecure_architecture']['crypto_architecture_infrastructure']['deprecated_libraries']:
                    findings.append(('insecure_architecture', 'crypto_architecture_infrastructure', module_name, MISUSE_PATTERNS['insecure_architecture']['crypto_architecture_infrastructure']['deprecated_libraries'][module_name], node.lineno))
            self.generic_visit(node)

        def visit_Call(self, node):
            if isinstance(node.func, ast.Attribute):
                if node.func.attr in MISUSE_PATTERNS['code_level_misuse']['weak_cryptography']:
                    findings.append(('code_level_misuse', 'weak_cryptography', node.func.attr, MISUSE_PATTERNS['code_level_misuse']['weak_cryptography'][node.func.attr], node.lineno))
                if node.func.attr in MISUSE_PATTERNS['code_level_misuse']['bad_randomness_handling']:
                    findings.append(('code_level_misuse', 'bad_randomness_handling', node.func.attr, MISUSE_PATTERNS['code_level_misuse']['bad_randomness_handling'][node.func.attr], node.lineno))
                if node.func.attr in MISUSE_PATTERNS['insecure_architecture']['iv_nonce_management']:
                    findings.append(('insecure_architecture', 'iv_nonce_management', node.func.attr, MISUSE_PATTERNS['insecure_architecture']['iv_nonce_management'][node.func.attr], node.lineno))
                if node.func.attr in MISUSE_PATTERNS['design_flaws']['program_design_flaws']:
                    findings.append(('design_flaws', 'program_design_flaws', node.func.attr, MISUSE_PATTERNS['design_flaws']['program_design_flaws'][node.func.attr], node.lineno))
            if isinstance(node.func, ast.Name):
                if node.func.id in MISUSE_PATTERNS['code_level_misuse']['weak_cryptography']:
                    findings.append(('code_level_misuse', 'weak_cryptography', node.func.id, MISUSE_PATTERNS['code_level_misuse']['weak_cryptography'][node.func.id], node.lineno))
                if node.func.id in MISUSE_PATTERNS['code_level_misuse']['bad_randomness_handling']:
                    findings.append(('code_level_misuse', 'bad_randomness_handling', node.func.id, MISUSE_PATTERNS['code_level_misuse']['bad_randomness_handling'][node.func.id], node.lineno))
                if node.func.id in MISUSE_PATTERNS['insecure_architecture']['iv_nonce_management']:
                    findings.append(('insecure_architecture', 'iv_nonce_management', node.func.id, MISUSE_PATTERNS['insecure_architecture']['iv_nonce_management'][node.func.id], node.lineno))
                if node.func.id in MISUSE_PATTERNS['design_flaws']['program_design_flaws']:
                    findings.append(('design_flaws', 'program_design_flaws', node.func.id, MISUSE_PATTERNS['design_flaws']['program_design_flaws'][node.func.id], node.lineno))
            self.generic_visit(node)

        def visit_Assign(self, node):
            for target in node.targets:
                if isinstance(target, ast.Name) and 'key' in target.id.lower():
                    findings.append(('code_level_misuse', 'coding_implementation_bugs', target.id, MISUSE_PATTERNS['code_level_misuse']['coding_implementation_bugs']['hardcoded_keys'], node.lineno))
            self.generic_visit(node)

    analyzer = CryptoAnalyzer()
    analyzer.visit(tree)

    return findings

# Function to print suggestions in a structured manner
def print_suggestions(findings):
    if not findings:
        logging.info("No cryptographic issues found.")
    else:
        logging.warning("Cryptographic Issues Found:")
        for finding in findings:
            logging.warning(f"Line {finding[4]}: {finding[2]} - {finding[3]} (Category: {finding[0]}, Subcategory: {finding[1]})")

# Analyze the sample code
findings = analyze_code(SAMPLE_CODE)
print_suggestions(findings)
